<a href="https://colab.research.google.com/github/Reethureddy8/DL-assign2-1/blob/main/Q1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.utils import to_categorical

# -------------------------------
# 🔹 Load and Preprocess Data
# -------------------------------
def load_dataset(path):
    df = pd.read_csv(path, sep='\t', header=None)
    df = df.dropna()
    input_seqs = df[1].astype(str).tolist()
    target_seqs = df[0].astype(str).apply(lambda x: '\t' + x + '\n').tolist()
    return input_seqs, target_seqs

train_inputs, train_targets = load_dataset('/content/hi.translit.sampled.train.tsv')

# Build character sets
input_chars = sorted(set(''.join(train_inputs)))
target_chars = sorted(set(''.join(train_targets)))

input_char2idx = {c: i for i, c in enumerate(input_chars)}
target_char2idx = {c: i for i, c in enumerate(target_chars)}

input_idx2char = {i: c for c, i in input_char2idx.items()}
target_idx2char = {i: c for c, i in target_char2idx.items()}

# Sequence lengths
input_len = max(len(seq) for seq in train_inputs)
target_len = max(len(seq) for seq in train_targets)


encoder_input = np.zeros((len(train_inputs), input_len, len(input_chars)), dtype='float32')
decoder_input = np.zeros((len(train_targets), target_len, len(target_chars)), dtype='float32')
decoder_output = np.zeros((len(train_targets), target_len, len(target_chars)), dtype='float32')

for i, (input_text, target_text) in enumerate(zip(train_inputs, train_targets)):
    for t, ch in enumerate(input_text):
        encoder_input[i, t, input_char2idx[ch]] = 1.0
    for t, ch in enumerate(target_text):
        decoder_input[i, t, target_char2idx[ch]] = 1.0
        if t > 0:
            decoder_output[i, t - 1, target_char2idx[ch]] = 1.0

# -------------------------------
# 🔹 Build Seq2Seq Model
# -------------------------------
hidden_units = 256

enc_inputs = Input(shape=(None, len(input_chars)))
enc_lstm = LSTM(hidden_units, return_state=True)
_, state_h, state_c = enc_lstm(enc_inputs)
enc_states = [state_h, state_c]

dec_inputs = Input(shape=(None, len(target_chars)))
dec_lstm = LSTM(hidden_units, return_sequences=True, return_state=True)
dec_outputs, _, _ = dec_lstm(dec_inputs, initial_state=enc_states)
dec_dense = Dense(len(target_chars), activation='softmax')
dec_outputs = dec_dense(dec_outputs)

model = Model([enc_inputs, dec_inputs], dec_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])


# -------------------------------
# 🔹 Train
# -------------------------------
model.fit(
    [encoder_input, decoder_input],
    decoder_output,
    batch_size=128,
    epochs=50,
    validation_split=0.2
)


Epoch 1/50
277/277 ━━━━━━━━━━━━━━━━━━━━ 94s 328ms/step - accuracy: 0.0595 - loss: 1.2118 - val_accuracy: 0.0618 - val_loss: 1.2832
Epoch 2/50
277/277 ━━━━━━━━━━━━━━━━━━━━ 91s 327ms/step - accuracy: 0.0704 - loss: 1.1340 - val_accuracy: 0.0622 - val_loss: 1.2950
Epoch 3/50
277/277 ━━━━━━━━━━━━━━━━━━━━ 143s 331ms/step - accuracy: 0.0759 - loss: 1.1104 - val_accuracy: 0.0644 - val_loss: 1.3003
Epoch 4/50
277/277 ━━━━━━━━━━━━━━━━━━━━ 140s 323ms/step - accuracy: 0.0828 - loss: 1.0797 - val_accuracy: 0.0646 - val_loss: 1.3011
Epoch 5/50
277/277 ━━━━━━━━━━━━━━━━━━━━ 90s 324ms/step - accuracy: 0.0878 - loss: 1.0753 - val_accuracy: 0.0621 - val_loss: 1.3069
Epoch 6/50
277/277 ━━━━━━━━━━━━━━━━━━━━ 144s 331ms/step - accuracy: 0.0964 - loss: 1.0423 - val_accuracy: 0.0625 - val_loss: 1.3232
Epoch 7/50
277/277 ━━━━━━━━━━━━━━━━━━━━ 142s 332ms/step - accuracy: 0.1041 - loss: 1.0155 - val_accuracy: 0.0625 - val_loss: 1.3109
Epoch 8/50
277/277 ━━━━━━━━━━━━━━━━━━━━ 138s 319ms/step - accuracy: 0.1117 - lo

In [5]:
# Inference: Encoder
encoder_model = Model(enc_inputs, enc_states)

# Inference: Decoder
dec_state_input_h = Input(shape=(hidden_units,))
dec_state_input_c = Input(shape=(hidden_units,))
dec_states_inputs = [dec_state_input_h, dec_state_input_c]

dec_inf_outputs, state_h_inf, state_c_inf = dec_lstm(
    dec_inputs, initial_state=dec_states_inputs
)
dec_states = [state_h_inf, state_c_inf]
dec_inf_outputs = dec_dense(dec_inf_outputs)

decoder_model = Model(
    [dec_inputs] + dec_states_inputs,
    [dec_inf_outputs] + dec_states
)


In [6]:
def transliterate_word(word):
    input_seq = np.zeros((1, input_len, len(input_chars)))
    for t, ch in enumerate(word):
        if ch in input_char2idx:
            input_seq[0, t, input_char2idx[ch]] = 1.0

    states = encoder_model.predict(input_seq)

    target_seq = np.zeros((1, 1, len(target_chars)))
    target_seq[0, 0, target_char2idx['\t']] = 1.0

    output = ''
    for _ in range(target_len):
        predictions, h, c = decoder_model.predict([target_seq] + states)
        idx = np.argmax(predictions[0, -1, :])
        char = target_idx2char[idx]
        if char == '\n':
            break
        output += char

        target_seq = np.zeros((1, 1, len(target_chars)))
        target_seq[0, 0, idx] = 1.0
        states = [h, c]

    return output


In [7]:
while True:
    test_input = input("Enter a Latin word (or type 'exit'): ")
    if test_input.lower() == 'exit':
        break
    print("Predicted Devanagari:", transliterate_word(test_input))


Enter a Latin word (or type 'exit'): ghar
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 256ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 222ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
Predicted Devanagari: घ
Enter a Latin word (or type 'exit'): me
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
Predicted Devanagari: म
Enter a Latin word (or type 'exit'): aam
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
Predicted Devanagari: आा
Enter a Latin word (or type 'exit'): dil
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
Predicted Devanagari: डि
Enter a Latin word (or type 'exit'): nadi
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━